In [ ]:
# SET CONFIG AND PARAMETERS
path = '/home/xilinx/jupyter_notebooks/qick/TII/characterizationScripts/zcu216/configuration/'
with open(path+"setConfig.py", 'r') as f:
    code1 = f.read()
    exec(code1)
with open(path+"setParametersSpinq10q69.py", 'r') as f:
    code2 = f.read()
    exec(code2)   

In [ ]:
class ResonatorSpectroscopyProgram(AveragerProgram):
    
    def ns2cycles(self, ns, gen_ch=None, ro_ch=None):
        return self.us2cycles(ns/1000, gen_ch=gen_ch, ro_ch=ro_ch)

    def initialize(self):
        cfg=self.cfg
        
        probe_ch    = cfg["probe_ch"]        
        feedback_ch = cfg["feedback_ch"]        
        ro_pulse_shape  = cfg["ro_pulse_shape"]
        ro_pulse_length = self.ns2cycles(cfg["ro_pulse_length"], gen_ch=probe_ch)  
        self.declare_gen(ch=probe_ch, 
                         nqz=cfg["probe_ch_nqz"], 
                         mux_freqs=self.cfg['ro_pulse_frequency'], 
                         mux_gains=self.cfg['ro_pulse_gain'])
        self.set_pulse_registers(ch=probe_ch, 
                                 style=ro_pulse_shape, 
                                 length=ro_pulse_length, 
                                 mask=[0,1,2,3])        
        for iCh, ch in enumerate(feedback_ch):
            self.declare_readout(ch=ch, 
                                 freq=self.cfg['acquisition_frequency'][iCh], 
                                 length = self.ns2cycles(self.cfg['acquisition_length'], ro_ch=ch), 
                                 gen_ch=probe_ch)
        self.synci(500)  # give processor some time to configure pulses
    
    def body(self):
        cfg=self.cfg
        probe_ch    = cfg["probe_ch"]
        feedback_ch = cfg["feedback_ch"]
        ro_pulse_start       = self.ns2cycles(cfg["ro_pulse_start"])     
        self.measure(pulse_ch= probe_ch, 
                     adcs = feedback_ch, 
                     adc_trig_offset=self.ns2cycles(cfg["delay_before_acquisition"]), 
                     pins = [0], 
                     wait=True, 
                     syncdelay=self.ns2cycles(cfg["relaxation_time"]), 
                     t=ro_pulse_start)

In [ ]:
def run_resonator_spectroscopy(frequency_range):
    fig = go.Figure()
    set_sweetspots()
    results=[]
    for frequency in tqdm(frequency_range):
        readout_cfg["ro_pulse_frequency"][0]    = frequency + readout_cfg["freq_original"][0]
        readout_cfg["acquisition_frequency"][0] = frequency + readout_cfg["freq_original"][0]
        readout_cfg["ro_pulse_frequency"][1]    = frequency + readout_cfg["freq_original"][1]
        readout_cfg["acquisition_frequency"][1] = frequency + readout_cfg["freq_original"][1]
        readout_cfg["ro_pulse_frequency"][2]    = frequency + readout_cfg["freq_original"][2]
        readout_cfg["acquisition_frequency"][2] = frequency + readout_cfg["freq_original"][2]
        readout_cfg["ro_pulse_frequency"][3]    = frequency + readout_cfg["freq_original"][3]
        readout_cfg["acquisition_frequency"][3] = frequency + readout_cfg["freq_original"][3]            
        program_cfg = {**channel_cfg,**readout_cfg} 
        program = ResonatorSpectroscopyProgram(soccfg=soc_cfg, cfg=program_cfg)
        results.append(program.acquire(soc))
    results = np.transpose(results)
    q1 = np.abs(results[0][0][0] + 1j*results[0][0][1])
    q2 = np.abs(results[0][1][0] + 1j*results[0][1][1]) 
    q3 = np.abs(results[0][2][0] + 1j*results[0][2][1]) 
    q4 = np.abs(results[0][3][0] + 1j*results[0][3][1]) 
    
    f1 = np.array(frequency_range + readout_cfg["freq_original"][0])
    f2 = np.array(frequency_range + readout_cfg["freq_original"][1])
    f3 = np.array(frequency_range + readout_cfg["freq_original"][2])
    f4 = np.array(frequency_range + readout_cfg["freq_original"][3])
    
    print("R1 min:", f1[q1.argmin()])
    print("R2 min:", f2[q2.argmin()])
    print("R3 min:", f3[q3.argmin()])
    print("R4 min:", f4[q4.argmin()])
    
    reset_bias()
    return f1,f2,f3,f4,q1,q2,q3,q4

In [ ]:
f1,f2,f3,f4,q1,q2,q3,q4 = run_resonator_spectroscopy(np.linspace(-2, 2, 250))

In [101]:
# Create a 2x2 subplot figure
fig = sp.make_subplots(
    rows=2, cols=2,
    subplot_titles=("Qubit 6", "Qubit 7", "Qubit 8", "Qubit 9"),
    x_title="Frequency (MHz)",
    y_title="Amplitude"
)

# Add each qubit's result to a different subplot
fig.add_trace(go.Scatter(x=f1, y=q1, mode='lines', name='Qubit 6'), row=1, col=1)
fig.add_trace(go.Scatter(x=f2, y=q2, mode='lines', name='Qubit 7'), row=1, col=2)
fig.add_trace(go.Scatter(x=f3, y=q3, mode='lines', name='Qubit 8'), row=2, col=1)
fig.add_trace(go.Scatter(x=f4, y=q4, mode='lines', name='Qubit 9'), row=2, col=2)

# Update layout with a main title
fig.update_layout(
    title="Resonator Spectroscopy",
    height = 800, width = 1000,
    title_x= 0.5,    
    showlegend=False
)

# Show the plot
fig.show()